# Bias Detection Engine Demo: Open Sentencing

In [1]:
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from server.routes.engine import *  # bias_table, bias_grid, consistency_table, filter_bias

The specified config.json file /Users/shoffman/Projects/External/embrace-call-for-code/bias-detection-engine/notebooks/server/config/mappings.json does not exist


In [2]:
df = pd.read_csv('../data/simulated_data_v0.4.csv', index_col=0).set_index(['Case ID', 'Race', 'Gender', 'Citizenship'])
fp = pd.concat([df.pop(c) for c in df.columns if c.startswith('FP')], axis=1)
sent = pd.concat([df.pop(c) for c in df.columns[-5:]], axis=1)
df

Alleged Crime Alleged Crime Category  \
Case ID Race     Gender Citizenship                                             
10000   Other    Female U.S. Citizen  Drug trafficking                 Felony   
10001   Hispanic Male   U.S. Citizen  Drug trafficking                 Felony   
10002   Black    Female U.S. Citizen  Drug trafficking                 Felony   
10003   White    Female U.S. Citizen  Drug trafficking                 Felony   
10004   White    Male   U.S. Citizen  Drug trafficking                 Felony   
...                                                ...                    ...   
70388   Other    Female U.S. Citizen  Drug trafficking                 Felony   
70389   Hispanic Female U.S. Citizen  Drug trafficking                 Felony   
70390   White    Female U.S. Citizen  Drug trafficking                 Felony   
70391   White    Male   U.S. Citizen  Drug trafficking                 Felony   
70392   Hispanic Female U.S. Citizen  Drug trafficking                 Felony   

                                     Alleged Crime Reported  \
Case ID Race     Gender Citizenship                           
10000   Other    Female U.S. Citizen          Investigation   
10001   Hispanic Male   U.S. Citizen          Investigation   
10002   Black    Female U.S. Citizen          Investigation   
10003   White    Female U.S. Citizen          Investigation   
10004   White    Male   U.S. Citizen          Investigation   
...                                                     ...   
70388   Other    Female U.S. Citizen          Investigation   
70389   Hispanic Female U.S. Citizen          Investigation   
70390   White    Female U.S. Citizen          Investigation   
70391   White    Male   U.S. Citizen          Investigation   
70392   Hispanic Female U.S. Citizen          Investigation   

                                                     Investigation  \
Case ID Race     Gender Citizenship                                  
10000   Other    Female U.S. Citizen  Unsolved or not arrested OUT   
10001   Hispanic Male   U.S. Citizen  Unsolved or not arrested OUT   
10002   Black    Female U.S. Citizen  Unsolved or not arrested OUT   
10003   White    Female U.S. Citizen                        Arrest   
10004   White    Male   U.S. Citizen                        Arrest   
...                                                            ...   
70388   Other    Female U.S. Citizen                        Arrest   
70389   Hispanic Female U.S. Citizen  Unsolved or not arrested OUT   
70390   White    Female U.S. Citizen                        Arrest   
70391   White    Male   U.S. Citizen                        Arrest   
70392   Hispanic Female U.S. Citizen                        Arrest   

                                             Arrest             Charges filed  \
Case ID Race     Gender Citizenship                                             
10000   Other    Female U.S. Citizen            NaN                       NaN   
10001   Hispanic Male   U.S. Citizen            NaN                       NaN   
10002   Black    Female U.S. Citizen            NaN                       NaN   
10003   White    Female U.S. Citizen  Charges filed  Court initial appearance   
10004   White    Male   U.S. Citizen   Released OUT                       NaN   
...                                             ...                       ...   
70388   Other    Female U.S. Citizen  Charges filed  Court initial appearance   
70389   Hispanic Female U.S. Citizen            NaN                       NaN   
70390   White    Female U.S. Citizen  Charges filed  Court initial appearance   
70391   White    Male   U.S. Citizen  Charges filed  Court initial appearance   
70392   Hispanic Female U.S. Citizen  Charges filed  Court initial appearance   

                                     Diversion  \
Case ID Race     Gender Citizenship              
10000   Other    Female U.S. Citizen       NaN   
10001   Hispanic Male   U.S. Citizen       NaN   
10

<img src="https://www.bjs.gov/content/downloadchart.cfm">

In [3]:
subset = df['Alleged Crime'] == 'Drug trafficking'
df = df.loc[subset, 'Alleged Crime Reported':]
fp = fp.loc[subset]
sent = sent.loc[subset]

In [4]:
bias_table(df, 'Race', 'defendants')

Closer to 1 is better. Disparate impact < 0.8 implies bias against the transition shown (red), > 1.25 implies bias for the transition (blue).

TODO: compare "life" and parole sentences separately

In [5]:
subset = df['Sentencing'].str.contains('Prison', case=False, na=False)
sent = sent.loc[subset & (sent['Estimated Sentence'] != 'life') & (sent['Given Sentence'] != 'life')].astype(int)
mean_difference(sent['Estimated Sentence'], sent['Given Sentence'], 'Race')

## Individual Fairness -- Fact Patterns

In [6]:
consistency_table(fp, df)